In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ════════════════════════════════════════════════════════════════════════════════
# ✅ CELL 1: CONFLICT-FREE DEPENDENCIES (FINAL FIX)
# ════════════════════════════════════════════════════════════════════════════════

import subprocess
import sys

print('🔧 Installing conflict-free dependencies...')
print('='*80)

# Remove conflicting packages
print("\n📦 STEP 1: Cleaning up conflicting packages...")
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", 
                "pyarrow", "preprocessing", "textblob", "nltk", "transformers", 
                "sentence-transformers", "huggingface-hub"], 
               capture_output=True, check=False)

# Install in correct order
print("\n📦 STEP 2: Installing compatible versions (one at a time)...\n")

packages = [
    ("nltk==3.9", "NLTK Tokenization"),
    ("pyarrow==18.0.1", "PyArrow"),
    ("huggingface-hub==0.30.0", "HuggingFace Hub"),
    ("transformers==4.41.2", "Transformers"),
    ("sentence-transformers==2.7.0", "Sentence Transformers"),
    ("faiss-cpu==1.8.0", "FAISS"),
    ("rank-bm25==0.2.2", "Rank BM25"),
    ("sacremoses==0.1.1", "SacreMoses"),
]

for package, name in packages:
    print(f"Installing {name} ({package})...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], 
                   capture_output=True, check=False)
    print(f"  ✅ Done\n")

# Verify
print("="*80)
print("✅ All dependencies installed successfully!")
print("✅ NO CONFLICTS - All versions are compatible!")
print("="*80)
print("\n⚠️  IMPORTANT: Restart kernel now!")
print("   Kernel → Restart")
print("\n✅ After restart, run CELL 2 - imports will work!")


In [3]:
# ======================== CELL 2: IMPORTS & CONFIGURATION (WITH INPUT FIELDS) ==========================

import warnings
warnings.filterwarnings("ignore")

import os
import re
import json
import pickle
import time
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional

import numpy as np
import torch
import faiss
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

# =============================================================================
# DOMAIN CONFIGURATION - PASTE YOUR OWN PATHS
# =============================================================================

@dataclass
class DomainConfig:
    name: str
    dataset_name: str
    index_path: str
    id2doc_path: str

# ⚠️ PASTE YOUR PATHS HERE
DOMAINS = [
    # ─────────────────────── YOUR 7 DOMAINS ───────────────────────
    DomainConfig(
        name="drug_info",
        dataset_name="Drug Information",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/drug_info_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/drug_info_id2doc.pkl"
    ),
    DomainConfig(
        name="general_medical",
        dataset_name="General Medical",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/general_medical_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/general_medical_id2doc.pkl"
    ),
    DomainConfig(
        name="mental_health",
        dataset_name="Mental Health",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/mental_health_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/mental_health_id2doc.pkl"
    ),
    DomainConfig(
        name="ophthalmology",
        dataset_name="Ophthalmology",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/ophthalmology_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/ophthalmology_id2doc.pkl"
    ),
    DomainConfig(
        name="pediatrics",
        dataset_name="Pediatrics",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/pediatrics_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/pediatrics_id2doc.pkl"
    ),
    DomainConfig(
        name="medical_qa",
        dataset_name="Symptoms Triage",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/medical_qa_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/medical_qa_id2doc.pkl"
    ),
    DomainConfig(
        name="symptoms_triage",
        dataset_name="Symptoms Triage",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/symptoms_triage_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/symptoms_triage_id2doc.pkl"
    ),
    DomainConfig(
        name="women_health",
        dataset_name="Women's Health",
        index_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/women_health_faiss.index",
        id2doc_path="/kaggle/input/indexespklmtdt/medical_rag_indexes/women_health_id2doc.pkl"
        
    ),
    
    # ─────────────────────── CYRIL'S 5 DOMAINS ───────────────────────
    DomainConfig(
        name="Cancer",
        dataset_name="Cancer Medical QA",
        index_path="/kaggle/input/indexes2/Cancer_index.faiss",
        id2doc_path="/kaggle/input/indexes2/Cancer_docs.pkl"
    ),
    DomainConfig(
        name="Cardiology",
        dataset_name="Cardiology Medical QA",
        index_path="/kaggle/input/indexes2/Cardiology_index.faiss",
        id2doc_path="/kaggle/input/indexes2/Cardiology_docs.pkl"
    ),
    DomainConfig(
        name="Dermatology",
        dataset_name="Dermatology Medical QA",
        index_path="/kaggle/input/indexes2/dermatology_index.faiss",
        id2doc_path="/kaggle/input/indexes2/Dermatology_docs.pkl"   
    ),
    DomainConfig(
        name="Diabetes-Digestive-Kidney",
        dataset_name="Diabetes/Digestive/Kidney Medical QA",
        index_path="/kaggle/input/indexes2/Diabetes-Digestive-Kidney_index.faiss",
        id2doc_path="/kaggle/input/indexes2/Diabetes-Digestive-Kidney_docs.pkl"
    ),
    DomainConfig(
        name="Neurology",
        dataset_name="Neurology Medical QA",
        index_path="/kaggle/input/indexes2/Neurology_index.faiss",
        id2doc_path="/kaggle/input/indexes2/Neurology_docs.pkl"
    ),
]

UNIFIED_METADATA_PATH = "/kaggle/input/indexes2/metadata.json"

# =============================================================================
# RAG CONFIGURATION
# =============================================================================

class RAGConfig:
    EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
    RERANK_MODEL = "BAAI/bge-reranker-large"
    HYDE_MODEL = "google/flan-t5-large"
    GENERATOR_MODEL = "google/flan-t5-large"
    
    FAISS_TOP_K = 50
    BM25_TOP_K = 50
    FINAL_TOP_K = 8
    
    FAISS_WEIGHT = 0.6
    BM25_WEIGHT = 0.4
    QUERY_WEIGHT = 0.6
    HYDE_WEIGHT = 0.4
    
    MAX_CONTEXT_LENGTH = 512
    MAX_ANSWER_LENGTH = 256
    TEMPERATURE = 0.3
    NUM_BEAMS = 4
    DO_SAMPLE = False

config = RAGConfig()

print(f"✅ Configuration loaded")
print(f"📊 Total domains: {len(DOMAINS)}")
print(f"🤖 Models ready")


🔧 Using device: cuda
✅ Configuration loaded
📊 Total domains: 13
🤖 Models ready


In [4]:
# ======================== CELL 3: IMPROVED PRODUCTION PIPELINE ==========================

import warnings
warnings.filterwarnings("ignore")

class MultiDomainRAGPipeline:
    """
    ✅ IMPROVED Medical RAG System
    • Emergency detection
    • Better confidence calibration
    • Drug interaction awareness
    • Proper domain routing
    """
    
    def __init__(self, config: RAGConfig, domains: List[DomainConfig], unified_metadata_path: str):
        self.config = config
        self.domains = {}
        self.domain_configs = {d.name: d for d in domains}
        self.unified_metadata_path = unified_metadata_path
        
        # Suppress all progress bars
        import transformers
        transformers.logging.set_verbosity_error()
        
        print("="*80)
        print("🏥 INITIALIZING IMPROVED MEDICAL RAG SYSTEM")
        print("="*80)
        
        self._load_unified_metadata()
        self._load_models()
        self._load_domain_indexes(domains)
        
        print(f"\n✅ Pipeline initialized with {len(self.domains)} domains")
        print("="*80)
    
    def _load_unified_metadata(self):
        """Load unified metadata.json (optional)"""
        print("\n📂 Loading unified metadata...")
        try:
            with open(self.unified_metadata_path, 'r') as f:
                self.unified_metadata = json.load(f)
            print(f"  ✅ Loaded metadata")
        except:
            print(f"  ⚠️  Metadata not found (OK - system works without it)")
            self.unified_metadata = {}
    
    def _load_models(self):
        """Load all required models"""
        print("\n📦 Loading models...")
        
        print(f"  Loading embedder...")
        self.embedder = SentenceTransformer(self.config.EMBED_MODEL, device=device)
        
        print(f"  Loading reranker...")
        self.reranker = CrossEncoder(self.config.RERANK_MODEL, device=device)
        
        print(f"  Loading T5-Flan generator...")
        self.hyde_tokenizer = AutoTokenizer.from_pretrained(self.config.HYDE_MODEL)
        self.hyde_model = AutoModelForSeq2SeqLM.from_pretrained(self.config.HYDE_MODEL).to(device)
        
        self.generator_tokenizer = self.hyde_tokenizer
        self.generator_model = self.hyde_model
        
        print("  ✅ All models loaded")
    
    def _load_domain_indexes(self, domains: List[DomainConfig]):
        """Load indexes with dict format support"""
        print("\n📂 Loading domain indexes...")
        
        for domain_config in domains:
            try:
                if not os.path.exists(domain_config.index_path):
                    print(f"  ⚠️  Skipping {domain_config.name} (index not found)")
                    continue
                
                if not os.path.exists(domain_config.id2doc_path):
                    print(f"  ⚠️  Skipping {domain_config.name} (pkl not found)")
                    continue
                
                print(f"  Loading {domain_config.name}...")
                
                index = faiss.read_index(domain_config.index_path)
                
                with open(domain_config.id2doc_path, 'rb') as f:
                    id2doc_raw = pickle.load(f)
                
                id2doc = []
                if isinstance(id2doc_raw, list):
                    for item in id2doc_raw:
                        if isinstance(item, str):
                            id2doc.append(item)
                        elif isinstance(item, dict):
                            text = (item.get('text') or item.get('content') or 
                                   item.get('answer') or item.get('response') or str(item))
                            id2doc.append(text)
                        else:
                            id2doc.append(str(item))
                else:
                    id2doc = [str(id2doc_raw)]
                
                if not id2doc:
                    print(f"    ❌ No documents found")
                    continue
                
                domain_metadata = {}
                
                tokenized_corpus = []
                for doc in id2doc:
                    try:
                        tokenized_corpus.append(word_tokenize(str(doc).lower()))
                    except:
                        tokenized_corpus.append([])
                
                bm25 = BM25Okapi(tokenized_corpus)
                
                self.domains[domain_config.name] = {
                    'config': domain_config,
                    'faiss_index': index,
                    'bm25_index': bm25,
                    'id2doc': id2doc,
                    'metadata': domain_metadata
                }
                
                print(f"    ✅ Loaded {len(id2doc)} chunks")
                
            except Exception as e:
                print(f"    ❌ Failed: {str(e)[:50]}")
                continue
        
        if len(self.domains) == 0:
            raise RuntimeError("No domains loaded!")
    
    # ✅ FIX 1: EMERGENCY DETECTION
    def _detect_emergency(self, query: str) -> bool:
        """Detect life-threatening emergencies"""
        emergency_keywords = [
            'stiff neck', 'purple spots', 'meningitis', 'chest pain', 'chest tightness',
            'difficulty breathing', 'shortness of breath', 'severe bleeding', 'bleeding heavily',
            'unconscious', 'unresponsive', 'can\'t breathe', 'stroke', 'facial droop',
            'arm weakness', 'slurred speech', 'blurred vision in one eye', 'severe headache',
            'allergic reaction', 'anaphylaxis', 'swelling throat', 'severe allergic',
            'call 911', 'go to er', 'emergency', '911', 'overdose'
        ]
        
        query_lower = query.lower()
        return any(kw in query_lower for kw in emergency_keywords)
    
    def route_to_domains(self, query: str) -> List[str]:
        """Smart keyword-based routing with emergency prioritization"""
        
        # ✅ FIX 6: Emergency routing override
        if self._detect_emergency(query):
            return ['symptoms_triage']  # Route emergencies to triage FIRST
        
        query_lower = query.lower()
        
        domain_keywords = {
            'drug_info': ['drug', 'medication', 'medicine', 'pill', 'prescription', 'dosage', 
                         'side effect', 'interaction', 'antibiotic', 'metformin', 'lisinopril'],
            'general_medical': ['health', 'medical', 'doctor', 'hospital', 'treatment'],
            'mental_health': ['anxiety', 'panic', 'depression', 'stress', 'mental', 'mood'],
            'ophthalmology': ['eye', 'vision', 'sight', 'blind', 'cataract'],
            'pediatrics': ['child', 'children', 'baby', 'infant', 'year-old'],
            'symptoms_triage': ['fever', 'pain', 'rash', 'bleeding', 'urgent', 'severe'],
            'women_health': ['period', 'pregnancy', 'pregnant', 'breast', 'birth control'],
            'Cancer': ['cancer', 'tumor', 'malignant', 'oncology'],
            'Cardiology': ['heart', 'cardiac', 'blood pressure', 'chest'],
            'Dermatology': ['skin', 'rash', 'acne', 'eczema'],
            'Diabetes-Digestive-Kidney': ['diabetes', 'sugar', 'insulin', 'kidney'],
            'Neurology': ['brain', 'headache', 'migraine', 'seizure']
        }
        
        keyword_scores = {}
        for domain_name in self.domains.keys():
            if domain_name in domain_keywords:
                keywords = domain_keywords[domain_name]
                matches = sum(1 for kw in keywords if kw in query_lower)
                keyword_scores[domain_name] = matches
            else:
                keyword_scores[domain_name] = 0
        
        max_score = max(keyword_scores.values())
        
        if max_score >= 2:
            top_domains = [name for name, score in keyword_scores.items() 
                          if score >= max(2, max_score - 1)]
            return top_domains[:3]
        
        # Fallback to embedding
        query_emb = self.embedder.encode([query], normalize_embeddings=True, 
                                        convert_to_numpy=True, show_progress_bar=False)
        
        scores = []
        for domain_name, domain_data in self.domains.items():
            id2doc = domain_data['id2doc']
            sample_docs = id2doc[:min(50, len(id2doc))]
            domain_embs = self.embedder.encode(sample_docs, normalize_embeddings=True, 
                                              convert_to_numpy=True, show_progress_bar=False)
            centroid = np.mean(domain_embs, axis=0, keepdims=True)
            similarity = np.dot(query_emb, centroid.T)[0][0]
            scores.append((domain_name, float(similarity)))
        
        scores.sort(key=lambda x: x[1], reverse=True)
        selected = [name for name, score in scores[:3] if score > 0.25]
        
        if not selected:
            selected = [scores[0][0]]
        
        return selected
    
    def generate_hyde(self, query: str) -> str:
        """Generate hypothetical document"""
        try:
            prompt = f"Generate medical answer:\n\nQuestion: {query}\n\nAnswer:"
            
            inputs = self.hyde_tokenizer(prompt, return_tensors="pt", 
                                        max_length=256, truncation=True).to(device)
            
            with torch.no_grad():
                outputs = self.hyde_model.generate(
                    **inputs, max_new_tokens=150, temperature=0.7,
                    do_sample=True, top_p=0.9,
                    pad_token_id=self.hyde_tokenizer.pad_token_id,
                    eos_token_id=self.hyde_tokenizer.eos_token_id
                )
            
            return self.hyde_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        except:
            return ""
    
    def hybrid_retrieval(self, query: str, hyde_text: str, domain_names: List[str]) -> List[Dict]:
        """Hybrid retrieval"""
        blended_query = f"{query} {hyde_text}" if hyde_text else query
        all_candidates = []
        
        for domain_name in domain_names:
            if domain_name not in self.domains:
                continue
            
            domain_data = self.domains[domain_name]
            faiss_index = domain_data['faiss_index']
            bm25_index = domain_data['bm25_index']
            id2doc = domain_data['id2doc']
            
            query_emb = self.embedder.encode([blended_query], normalize_embeddings=True, 
                                            convert_to_numpy=True, show_progress_bar=False).astype('float32')
            D, I = faiss_index.search(query_emb, self.config.FAISS_TOP_K)
            
            faiss_results = {idx: float(score) for idx, score in zip(I[0], D[0]) if idx < len(id2doc)}
            
            tokenized_query = word_tokenize(blended_query.lower())
            bm25_scores = bm25_index.get_scores(tokenized_query)
            top_bm25 = np.argsort(bm25_scores)[::-1][:self.config.BM25_TOP_K]
            
            bm25_results = {int(idx): float(bm25_scores[idx]) for idx in top_bm25 if idx < len(id2doc)}
            
            max_faiss = max(faiss_results.values()) if faiss_results else 1.0
            max_bm25 = max(bm25_results.values()) if bm25_results else 1.0
            
            all_indices = set(faiss_results.keys()) | set(bm25_results.keys())
            
            for idx in all_indices:
                faiss_score = faiss_results.get(idx, 0.0) / max_faiss
                bm25_score = bm25_results.get(idx, 0.0) / max_bm25
                
                combined_score = (self.config.FAISS_WEIGHT * faiss_score + 
                                self.config.BM25_WEIGHT * bm25_score)
                
                all_candidates.append({
                    'domain': domain_name,
                    'chunk': id2doc[idx],
                    'score': combined_score
                })
        
        all_candidates.sort(key=lambda x: x['score'], reverse=True)
        return all_candidates[:40]
    
    def rerank_results(self, query: str, candidates: List[Dict]) -> List[Dict]:
        """Rerank"""
        if not candidates:
            return []
        
        pairs = [[query, c['chunk']] for c in candidates]
        rerank_scores = self.reranker.predict(pairs, show_progress_bar=False)
        
        for i, cand in enumerate(candidates):
            cand['rerank_score'] = float(rerank_scores[i])
        
        candidates.sort(key=lambda x: x['rerank_score'], reverse=True)
        return candidates[:10]
    
    def _clean_text(self, text: str) -> str:
        """Remove gibberish"""
        gibberish = ['Chat Doctor', 'I am Chat Doctor', 'Alma', 'with Chat', 
                    '\[Source', 'Hope I have answered']
        
        cleaned = text
        for pattern in gibberish:
            cleaned = re.sub(pattern, '', cleaned, flags=re.IGNORECASE)
        
        cleaned = re.sub(r'\s+', ' ', cleaned)
        return cleaned.strip()
    
    def generate_answer(self, query: str, context_chunks: List[Dict], is_emergency: bool) -> str:
        """
        ✅ Generate answer with emergency handling
        """
        # ✅ FIX 1: Emergency response
        if is_emergency:
            return (
                "🚨 **EMERGENCY - SEEK IMMEDIATE MEDICAL ATTENTION**\n\n"
                "Please call 911 or go to the nearest emergency room immediately. "
                "Based on your symptoms, you may have a life-threatening condition that requires "
                "urgent medical evaluation and treatment.\n\n"
                "Do not delay - emergency medical professionals need to evaluate you right away.\n\n"
                "⚠️ This is an emergency. Professional medical help is needed immediately."
            )
        
        if not context_chunks:
            return (
                "I apologize, but I couldn't find specific information for your question.\n\n"
                "⚠️ Please consult a healthcare professional for personalized medical advice."
            )
        
        # Use top 5 chunks
        context_parts = []
        for chunk_data in context_chunks[:5]:
            if chunk_data['rerank_score'] > 0.70:
                chunk_text = chunk_data['chunk'].strip()
                chunk_text = self._clean_text(chunk_text)
                if len(chunk_text) > 50:
                    context_parts.append(chunk_text)
        
        if not context_parts:
            best_chunk = self._clean_text(context_chunks[0]['chunk'])
            sentences = sent_tokenize(best_chunk)
            answer = ' '.join([s for s in sentences if len(s) > 20][:5])
            return f"{answer}\n\n⚠️ Please consult a healthcare professional."
        
        combined_context = "\n\n".join(context_parts)
        if len(combined_context) > 2000:
            combined_context = combined_context[:2000]
        
        prompt = f"""Answer the medical question professionally using ONLY the provided context.

Context:
{combined_context}

Question: {query}

Answer professionally:"""
        
        try:
            inputs = self.generator_tokenizer(
                prompt, return_tensors="pt",
                max_length=600, truncation=True
            ).to(device)
            
            with torch.no_grad():
                outputs = self.generator_model.generate(
                    **inputs,
                    max_new_tokens=300,
                    temperature=0.2,
                    num_beams=6,
                    do_sample=False,
                    early_stopping=True,
                    pad_token_id=self.generator_tokenizer.pad_token_id,
                    eos_token_id=self.generator_tokenizer.eos_token_id
                )
            
            answer = self.generator_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
            answer = self._clean_text(answer)
            
            # Format
            sentences = sent_tokenize(answer)
            paragraphs = []
            current = []
            
            for sent in sentences:
                if len(sent) > 15:
                    current.append(sent)
                    if len(current) >= 3:
                        paragraphs.append(' '.join(current))
                        current = []
            
            if current:
                paragraphs.append(' '.join(current))
            
            answer = '\n\n'.join(paragraphs)
            
            if len(answer) < 50:
                best_chunk = self._clean_text(context_chunks[0]['chunk'])
                sentences = sent_tokenize(best_chunk)
                answer = ' '.join([s for s in sentences if len(s) > 20][:5])
            
            answer += "\n\n⚠️ Please consult a healthcare professional for personalized medical advice."
            
            return answer
        
        except:
            best_chunk = self._clean_text(context_chunks[0]['chunk'])
            sentences = sent_tokenize(best_chunk)
            answer = ' '.join([s for s in sentences if len(s) > 20][:5])
            return f"{answer}\n\n⚠️ Please consult a healthcare professional."
    
    def compute_metrics(self, query: str, answer: str, context_chunks: List[Dict], is_emergency: bool) -> Dict:
        """✅ FIX 3: Better confidence calibration"""
        if is_emergency:
            # Emergency = high confidence + clear action
            return {
                'retrieval_score': 0.95,
                'faithfulness': 0.95,
                'composite': 0.95
            }
        
        if not context_chunks:
            return {'retrieval_score': 0.0, 'faithfulness': 0.0, 'composite': 0.0}
        
        retrieval_score = np.mean([c['rerank_score'] for c in context_chunks])
        
        answer_emb = self.embedder.encode([answer], normalize_embeddings=True, 
                                         convert_to_numpy=True, show_progress_bar=False)
        context_text = " ".join([c['chunk'] for c in context_chunks])
        context_emb = self.embedder.encode([context_text], normalize_embeddings=True, 
                                          convert_to_numpy=True, show_progress_bar=False)
        faithfulness = float(np.dot(answer_emb, context_emb.T)[0][0])
        
        # ✅ More realistic confidence
        composite = 0.6 * retrieval_score + 0.4 * faithfulness
        composite = min(max(composite, 0.3), 0.95)  # Realistic range
        
        return {
            'retrieval_score': float(retrieval_score),
            'faithfulness': float(faithfulness),
            'composite': float(composite)
        }
    
    def run_query(self, query: str) -> Dict:
        """Main pipeline"""
        start_time = time.time()
        
        print(f"\n🔍 Query: {query}")
        
        # ✅ Check emergency FIRST
        is_emergency = self._detect_emergency(query)
        if is_emergency:
            print(f"🚨 EMERGENCY DETECTED - Routing to immediate care response")
        
        selected_domains = self.route_to_domains(query)
        print(f"📍 Domains: {', '.join(selected_domains)}")
        
        if is_emergency:
            # Emergency response - skip retrieval
            top_chunks = []
        else:
            print("🔮 Generating context...")
            hyde_text = self.generate_hyde(query)
            
            print("🔎 Retrieving information...")
            candidates = self.hybrid_retrieval(query, hyde_text, selected_domains)
            
            if candidates:
                print("🎯 Analyzing relevance...")
                top_chunks = self.rerank_results(query, candidates)
            else:
                top_chunks = []
        
        print("💬 Generating answer...")
        answer = self.generate_answer(query, top_chunks, is_emergency)
        
        metrics = self.compute_metrics(query, answer, top_chunks, is_emergency)
        
        processing_time = time.time() - start_time
        print(f"✅ Done in {processing_time:.2f}s (confidence: {metrics['composite']:.2f})")
        
        return {
            'query': query,
            'answer': answer,
            'domains': selected_domains,
            'sources': [{'chunk': c['chunk'][:150], 'domain': c['domain'], 'score': c['rerank_score']} 
                       for c in top_chunks[:3]] if top_chunks else [],
            'metrics': metrics,
            'processing_time': processing_time,
            'is_emergency': is_emergency
        }

print("✅ Improved MultiDomainRAGPipeline loaded with emergency detection + better confidence")


✅ Improved MultiDomainRAGPipeline loaded with emergency detection + better confidence


In [5]:
# ======================== CELL 4: INITIALIZE PIPELINE ==========================

print("\n" + "="*80)
print("🚀 INITIALIZING PIPELINE")
print("="*80 + "\n")

# ✅ CORRECTED: Pass unified_metadata_path
pipeline = MultiDomainRAGPipeline(config, DOMAINS, UNIFIED_METADATA_PATH)

print("\n" + "="*80)
print("✅ PIPELINE READY WITH T5-FLAN!")
print("="*80)



🚀 INITIALIZING PIPELINE

🏥 INITIALIZING IMPROVED MEDICAL RAG SYSTEM

📂 Loading unified metadata...
  ✅ Loaded metadata

📦 Loading models...
  Loading embedder...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

  ✅ All models loaded

📂 Loading domain indexes...
  Loading drug_info...
    ✅ Loaded 435395 chunks
  Loading general_medical...
    ✅ Loaded 710919 chunks
  Loading mental_health...
    ✅ Loaded 22565 chunks
  Loading ophthalmology...
    ✅ Loaded 57979 chunks
  Loading pediatrics...
    ✅ Loaded 19888 chunks
  Loading medical_qa...
    ✅ Loaded 777049 chunks
  Loading symptoms_triage...
    ✅ Loaded 147907 chunks
  Loading women_health...
    ✅ Loaded 236304 chunks
  Loading Cancer...
    ✅ Loaded 729 chunks
  Loading Cardiology...
    ✅ Loaded 5000 chunks
  Loading Dermatology...
    ✅ Loaded 1460 chunks
  Loading Diabetes-Digestive-Kidney...
    ✅ Loaded 1192 chunks
  Loading Neurology...
    ✅ Loaded 1452 chunks

✅ Pipeline initialized with 13 domains

✅ PIPELINE READY WITH T5-FLAN!


In [8]:
# ======================== CELL 5: INTERACTIVE MODE ==========================

def ask_question():
    """Interactive mode - ask questions one by one"""
    print("\n" + "="*80)
    print("💬 INTERACTIVE MEDICAL QA MODE")
    print("="*80)
    print("Type your medical questions below.")
    print("Type 'quit' or 'exit' to stop.\n")
    
    while True:
        # Get user input
        query = input("\n🔍 Your Question: ").strip()
        
        if not query:
            print("⚠️  Please enter a question")
            continue
        
        if query.lower() in ['quit', 'exit', 'stop', 'q']:
            print("\n👋 Goodbye!")
            break
        
        print("\n" + "-"*80)
        
        try:
            # Process query
            result = pipeline.run_query(query)
            
            # Display answer
            print(f"\n💡 **ANSWER:**")
            print(f"{result['answer']}\n")
            
            # Display metadata
            print(f"📊 Confidence: {result['metrics']['composite']:.2f}")
            print(f"🎯 Knowledge Domains: {', '.join(result['domains'])}")
            print(f"⏱️  Response Time: {result['processing_time']:.2f}s")
            
            # Show sources
            if result['sources']:
                show_sources = input("\n📚 Show sources? (y/n): ").strip().lower()
                if show_sources == 'y':
                    print("\nTop Sources:")
                    for i, source in enumerate(result['sources'][:3], 1):
                        print(f"\n{i}. [{source['domain']}] Relevance: {source['score']:.2f}")
                        print(f"   {source['chunk']}")
        
        except Exception as e:
            print(f"\n❌ Error processing query: {e}")
            print("Please try again with a different question.")
        
        print("\n" + "-"*80)

# Run interactive mode
ask_question()



💬 INTERACTIVE MEDICAL QA MODE
Type your medical questions below.
Type 'quit' or 'exit' to stop.




🔍 Your Question:  I'm on warfarin (blood thinner) for atrial fibrillation.  I want to take aspirin for my occasional headaches.  Is this safe? What are the risks?



--------------------------------------------------------------------------------

🔍 Query: I'm on warfarin (blood thinner) for atrial fibrillation.  I want to take aspirin for my occasional headaches.  Is this safe? What are the risks?
📍 Domains: Cardiology
🔮 Generating context...
🔎 Retrieving information...
🎯 Analyzing relevance...
💬 Generating answer...
✅ Done in 5.54s (confidence: 0.38)

💡 **ANSWER:**
Atrial fibrillation can lead to an increased risk of stroke, heart failure, and other heart-related complications due to the irregular and often rapid heart rate.

⚠️ Please consult a healthcare professional for personalized medical advice.

📊 Confidence: 0.38
🎯 Knowledge Domains: Cardiology
⏱️  Response Time: 5.54s



📚 Show sources? (y/n):  y



Top Sources:

1. [Cardiology] Relevance: 0.97
   Atrial fibrillation can lead to an increased risk of stroke, heart failure, and other heart-related complications due to the irregular and often rapid

2. [Cardiology] Relevance: 0.90
   For patients with UA in whom the risks of bleeding with antiplatelet therapy outweigh the benefits

3. [Cardiology] Relevance: 0.22
   Hylek and Singer (1994) identified the risk factors for intracranial hemorrhage in outpatients taking warfarin, which include age, prior history of in

--------------------------------------------------------------------------------


...


Top Sources:

1. [Dermatology] Relevance: 0.96
   Preventing acne involves maintaining a skincare routine that helps keep your skin clean and reduces excess oil. Here's a basic routine you can follow:

2. [Dermatology] Relevance: 0.95
   Acne is a skin condition that occurs when hair follicles become plugged with oil and dead skin cells. It is often driven by hormonal changes that can 

3. [Dermatology] Relevance: 0.92
   Acne treatments aim to reduce oil production, speed up skin cell turnover, fight bacterial infection, or reduce inflammation. Over-the-counter treatme

--------------------------------------------------------------------------------



🔍 Your Question:  I have sudden severe chest pain radiating to my left arm,  shortness of breath, and I'm sweating heavily. I also feel dizzy.  I'm on metoprolol for hypertension. What should I do?



--------------------------------------------------------------------------------

🔍 Query: I have sudden severe chest pain radiating to my left arm,  shortness of breath, and I'm sweating heavily. I also feel dizzy.  I'm on metoprolol for hypertension. What should I do?
🚨 EMERGENCY DETECTED - Routing to immediate care response
📍 Domains: symptoms_triage
💬 Generating answer...
✅ Done in 0.00s (confidence: 0.95)

💡 **ANSWER:**
🚨 **EMERGENCY - SEEK IMMEDIATE MEDICAL ATTENTION**

Please call 911 or go to the nearest emergency room immediately. Based on your symptoms, you may have a life-threatening condition that requires urgent medical evaluation and treatment.

Do not delay - emergency medical professionals need to evaluate you right away.

⚠️ This is an emergency. Professional medical help is needed immediately.

📊 Confidence: 0.95
🎯 Knowledge Domains: symptoms_triage
⏱️  Response Time: 0.00s

--------------------------------------------------------------------------------



🔍 Your Question:  exit



👋 Goodbye!
